In [1]:
!pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/3e/d3/309769dad11d5f75b81c7252d9dc849ed440d0921215e759af169054f3b6/openai-1.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/a2/65/6940eeb21dcb2953778a6895281c179efd9100463ff08cb6232bb6480da7/httpx-0.25.2-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore==1.* from https://files.pythonhosted.org/packages/56/ba/78b0a99c4da0ff8b0f59defa2f13ca4668189b134bd9840b6202a93d9a0f/httpcore-1.0.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.1 MB/s eta 0:00:00


In [2]:
import os
import openai
import json

In [3]:
from openai import OpenAI

api_key  = 'Your-API-Key'

client = OpenAI(api_key=api_key)


## Simple example

In [4]:
# Say hello and get a response

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


## Function Calling Example

In [5]:
# Define a dummy function

def get_product_of_two_numbers(num1, num2):
    
    answer = str(num1 * num2)
    
    return answer

In [6]:
tools = [
  {
      
    "type": "function", # What kind of tool is it
    "function": {
      "name": "get_product_of_two_numbers",
      "description": "Mutiplies two numbers and returns an answer.",
        # What do we want chatgpt to return
      "parameters": {
        "type": "object", # We want an object
          
        "properties": {
          "first_number": {
            "type": "integer",
            "description": "The first number e.g. 3",
          },
          "second_number": {
            "type": "integer",
            "description": "The first number e.g. 5",
          }   
        }, # close properties
          
        # Both numbers are required
        "required": ["first_number", "second_number"],
          
      } # close parameters
          
    } # close function
        
  } # close first curly brace
]

In [7]:
# Send a message to chatgpt

messages = [{"role": "user", "content": "What is the product of 5 and 2?"}]
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)

In [8]:
print(completion)

ChatCompletion(id='chatcmpl-8SiuOFmBGCCc9o4y6Qbyv80t71dFU', choices=[Choice(finish_reason='tool_calls', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hL7ePCJaBTbNh1QPifVmXZAa', function=Function(arguments='{\n  "first_number": 5,\n  "second_number": 2\n}', name='get_product_of_two_numbers'), type='function')]))], created=1701856504, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=89, total_tokens=116))


In [9]:
# Get the reason for the finish
completion.choices[0].finish_reason

'tool_calls'

In [10]:
# Get the function name
# Note the this is important because there could be many functions
function_name = completion.choices[0].message.tool_calls[0].function.name

function_name

'get_product_of_two_numbers'

In [11]:
completion.choices[0].message.tool_calls[0].function.arguments

'{\n  "first_number": 5,\n  "second_number": 2\n}'

In [12]:
arguments = completion.choices[0].message.tool_calls[0].function.arguments

arguments

'{\n  "first_number": 5,\n  "second_number": 2\n}'

In [13]:
# Convert to json format

json_arguments = json.loads(arguments)

json_arguments 

{'first_number': 5, 'second_number': 2}

In [14]:
# Get the arguments that chatgpt has provided 

print(json_arguments['first_number'])
print(json_arguments['second_number'])

5
2


In [15]:
# Call the function

num1 = json_arguments['first_number']
num2 = json_arguments['second_number']

answer = get_product_of_two_numbers(num1, num2)

answer

'10'

In [16]:
# We add a new messages to the messages list and send it
# to chatgpt

messages = [{"role": "user", "content": "What is the product of 5 and 2?"},
           {'role': 'function', 'name': function_name, 'content': answer}]

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)

In [17]:
completion

ChatCompletion(id='chatcmpl-8SiuQWl04YHQhRIJ2TXMJCwmv4QLw', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The product of 5 and 2 is 10.', role='assistant', function_call=None, tool_calls=None))], created=1701856506, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=98, total_tokens=111))

In [18]:
# This is the final response from Chatgpt that we show to the user

completion.choices[0].message.content

'The product of 5 and 2 is 10.'